In [1]:
import io

**Q1**

In [2]:
! pip install gdown

In [3]:
import torch
import numpy as np
import pandas as pd
import torchvision
import torch.nn as nn # use nn functions like sigmoid, ReLu, softmax
import torchvision.transforms as transforms # use for transformerin on data
import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
import torchvision.models as models
import os
from PIL import Image
import io
import zipfile
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
import time
from scipy.spatial.distance import pdist, squareform
from sklearn.decomposition import PCA
import json
from tqdm import tqdm

In [4]:
! gdown 1hAfkobN0YBk2D2fqhhRht2yKmF1nNwWz

Downloading...
From: https://drive.google.com/uc?id=1hAfkobN0YBk2D2fqhhRht2yKmF1nNwWz
To: /kaggle/working/kaggle.json
100%|█████████████████████████████████████████| 64.0/64.0 [00:00<00:00, 340kB/s]


In [5]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d danielbacioiu/tig-aluminium-5083

Dataset URL: https://www.kaggle.com/datasets/danielbacioiu/tig-aluminium-5083
License(s): CC-BY-SA-4.0
100%|██████████████████████████████████████▉| 11.2G/11.2G [00:45<00:00, 280MB/s]
100%|███████████████████████████████████████| 11.2G/11.2G [00:45<00:00, 263MB/s]


In [20]:
! pip install torchsummary

In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import zipfile
import json
import io
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Download and set up Kaggle credentials (if needed)
# Note: Adjust your download and setup steps here accordingly
! gdown 1hAfkobN0YBk2D2fqhhRht2yKmF1nNwWz
! mkdir -p ~/.kaggle
! mv kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

# Download dataset (assuming you've adjusted the download commands)
! kaggle datasets download -d danielbacioiu/tig-aluminium-5083

# Define transformations for image preprocessing
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

class SMDataset(Dataset):
    def __init__(self, zip_file_path, folder_name, json_file_path, transform=None):
        self.zip_file = zipfile.ZipFile(zip_file_path, 'r')
        self.folder_name = folder_name.rstrip('/') + '/'
        self.file_list = [name for name in self.zip_file.namelist() if name.startswith(self.folder_name) and name.endswith('.png')]
        self.transform = transform

        # Load labels from JSON file
        with self.zip_file.open(json_file_path) as json_file:
            self.labels_dict = json.load(json_file)

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        try:
            with self.zip_file.open(self.file_list[idx]) as file:
                image = Image.open(io.BytesIO(file.read())).convert('RGB')
                if self.transform:
                    image = self.transform(image)

            # Extract the relevant part of the filename for lookup
            file_name = self.file_list[idx].replace(self.folder_name, '')

            # Get the label for the current file
            label = self.labels_dict.get(file_name, -1)

            return image, label
        except Exception as e:
            print(f"Error loading item {idx}: {e}")
            return None, None

# Define paths for train and test JSON files
train_json_path = 'al5083/train/train.json'
test_json_path = 'al5083/test/test.json'

# Create dataset instances
total_train_dataset = SMDataset(
    zip_file_path='/kaggle/working/tig-aluminium-5083.zip',
    folder_name='al5083/train',
    json_file_path=train_json_path,
    transform=transform
)

total_test_dataset = SMDataset(
    zip_file_path='/kaggle/working/tig-aluminium-5083.zip',
    folder_name='al5083/test',
    json_file_path=test_json_path,
    transform=transform
)

# Split datasets into train, validation, and additional 90% train datasets
total_train_size = len(total_train_dataset)
total_test_size = len(total_test_dataset)

train_size = int(0.8 * total_train_size)
valid_size = int(0.5 * total_train_size)
train90_size = total_train_size - train_size - valid_size

test_size = int(0.1 * total_test_size)
test90_size = total_test_size - test_size

train_dataset, valid_dataset, train90_dataset = random_split(total_train_dataset, [train_size, valid_size, train90_size])
test_dataset, test90_dataset = random_split(total_test_dataset, [test_size, test90_size])

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=1, shuffle=True)

# Define MyModel in PyTorch
class MyModel(nn.Module):
    def __init__(self, num_classes=6, l2_reg=0.01):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 256, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(p=0.2)

        self.conv2 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout2 = nn.Dropout(p=0.2)

        self.conv3 = nn.Conv2d(128, 64, kernel_size=3)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(64 * 31 * 31, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = self.dropout1(x)

        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = self.dropout2(x)

        x = F.relu(self.conv3(x))
        x = self.pool3(x)

        x = self.flatten(x)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

# Initialize model and move to appropriate device
model = MyModel().to(device)

# Print model summary
from torchsummary import summary
summary(model, input_size=(3, 256, 256))

# Count total number of parameters in the model
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total number of parameters: {total_params}")

# Define loss function and optimizer
lossfn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0.0
    train_batches = len(train_dataloader)
    with tqdm(total=train_batches, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as pbar:
        for i, (images, labels) in enumerate(train_dataloader):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = lossfn(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            if i % 16 == 0:
                pbar.set_postfix({'loss': train_loss / (i + 1)})
                pbar.update(16)

    # Validation phase
    model.eval()
    correct = 0
    total = 0
    valid_batches = len(valid_dataloader)
    with tqdm(total=valid_batches, desc=f'Validation {epoch+1}/{num_epochs}', unit='batch') as pbar:
        with torch.no_grad():
            for images, labels in valid_dataloader:
                images = images.to(device)
                labels = labels.to(device)

                # Forward pass
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                pbar.update(1)

        accuracy = 100 * correct / total
        print(f'Accuracy of the network on the validation images: {accuracy:.2f} %')



Downloading...
From: https://drive.google.com/uc?id=1hAfkobN0YBk2D2fqhhRht2yKmF1nNwWz
To: /kaggle/working/kaggle.json
100%|█████████████████████████████████████████| 64.0/64.0 [00:00<00:00, 380kB/s]
Dataset URL: https://www.kaggle.com/datasets/danielbacioiu/tig-aluminium-5083
License(s): CC-BY-SA-4.0
tig-aluminium-5083.zip: Skipping, found more recently modified local copy (use --force to force download)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [-1, 256, 256, 256]           7,168
         MaxPool2d-2        [-1, 256, 128, 128]               0
           Dropout-3        [-1, 256, 128, 128]               0
            Conv2d-4        [-1, 128, 128, 128]         295,040
         MaxPool2d-5          [-1, 128, 64, 64]               0
           Dropout-6          [-1, 128, 64, 64]               0
            Conv2d-7           [-1, 64, 62, 62]          73,792
         MaxPoo

Epoch 1/20: 1344batch [07:26,  3.01batch/s, loss=0.669]                       
Validation 1/20: 100%|██████████| 5334/5334 [01:32<00:00, 57.46batch/s]


Accuracy of the network on the validation images: 86.50 %


Epoch 2/20: 1344batch [07:26,  3.01batch/s, loss=0.195]                       
Validation 2/20: 100%|██████████| 5334/5334 [01:32<00:00, 57.42batch/s]


Accuracy of the network on the validation images: 90.57 %


...

Accuracy of the network on the validation images: 99.64 %


Epoch 19/20: 1344batch [07:25,  3.02batch/s, loss=0.0136]                       
Validation 19/20: 100%|██████████| 5334/5334 [01:32<00:00, 57.94batch/s]


Accuracy of the network on the validation images: 99.25 %


Epoch 20/20: 1344batch [07:23,  3.03batch/s, loss=0.0148]                       
Validation 20/20: 100%|██████████| 5334/5334 [01:32<00:00, 57.71batch/s]

Accuracy of the network on the validation images: 99.76 %


In [28]:
correct = 0
total = 0
for images,labels in test_dataloader:
  images = images.to(device)
  labels = labels.to(device)
  outputs = model(images)
  prediction = torch.argmax(outputs, dim = 1)
  correct += (prediction == labels).sum().item()
  total += labels.size(0)

print(correct/total * 100)

41.33738601823708


In [30]:
import time
import torch

def calculate_fps(model, dataloader, device, num_batches=10):
    model.eval()  # Set the model to evaluation mode
    
    total_time = 0.0
    total_images = 0
    
    with torch.no_grad():
        for i, (images, labels) in enumerate(dataloader):
            if i >= num_batches:
                break
            
            images = images.to(device)
            total_images += images.size(0)
            
            start_time = time.time()
            outputs = model(images)
            end_time = time.time()
            
            total_time += (end_time - start_time)
    
    fps = total_images / total_time
    return fps

fps = calculate_fps(model, valid_dataloader, device, num_batches=10)
print(f'FPS: {fps:.2f}')  # Corrected indentation here

FPS: 1530.99


**Q2**

In [31]:
import numpy as np

# Example image (5x5)
image = np.array([[0.51, 0.90, 0.88, 0.84, 0.05],
                  [0.40, 0.62, 0.22, 0.59, 0.10],
                  [0.11, 0.20, 0.74, 0.33, 0.14],
                  [0.47, 0.01, 0.85, 0.70, 0.09],
                  [0.76, 0.19, 0.72, 0.17, 0.57]])

# Corrected kernel (2x2)
kernel_1 = np.array([[-0.13, 0.15], [-0.51, 0.62]])

# Convolution function
def convolve(image, kernel, stride=1, padding=0):
    kernel_height, kernel_width = kernel.shape
    image_height, image_width = image.shape

    # Apply padding
    if padding > 0:
        image = np.pad(image, ((padding, padding), (padding, padding)), mode='constant')

    # Determine output dimensions
    out_height = (image_height - kernel_height) // stride + 1
    out_width = (image_width - kernel_width) // stride + 1

    # Initialize output
    output = np.zeros((out_height, out_width))

    # Perform convolution
    for y in range(out_height):
        for x in range(out_width):
            output[y, x] = np.sum(image[y*stride:y*stride+kernel_height, x*stride:x*stride+kernel_width] * kernel)

    return output

# First convolution output
conv_output_1 = convolve(image, kernel_1)
print("First Convolution Output:\n", conv_output_1)

First Convolution Output:
 [[ 0.2491 -0.1648  0.2652 -0.3406]
 [ 0.1089  0.3092 -0.1129 -0.1432]
 [-0.2178  0.6069 -0.0462 -0.3231]
 [-0.3294  0.4757 -0.2673  0.1892]]


In [32]:
def max_pooling(feature_map, size=2, stride=2):
    pooled_height = (feature_map.shape[0] - size) // stride + 1
    pooled_width = (feature_map.shape[1] - size) // stride + 1

    pooled_output = np.zeros((pooled_height, pooled_width))

    for y in range(pooled_height):
        for x in range(pooled_width):
            pooled_output[y, x] = np.max(feature_map[y*stride:y*stride+size, x*stride:x*stride+size])

    return pooled_output

# Max-pooling output
pooled_output = max_pooling(conv_output_1)
print("Max-Pooling Output:\n", pooled_output)

Max-Pooling Output:
 [[0.3092 0.2652]
 [0.6069 0.1892]]


In [33]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Sigmoid output
sigmoid_output = sigmoid(pooled_output)
print("Sigmoid Output:\n", sigmoid_output)

Sigmoid Output:
 [[0.57668998 0.56591413]
 [0.64723333 0.5471594 ]]


In [34]:
import numpy as np

# Given image and kernel
image = np.array([[0.51, 0.90, 0.88, 0.84, 0.05],
                  [0.40, 0.62, 0.22, 0.59, 0.10],
                  [0.11, 0.20, 0.74, 0.33, 0.14],
                  [0.47, 0.01, 0.85, 0.70, 0.09],
                  [0.76, 0.19, 0.72, 0.17, 0.57]])

kernel = np.array([[-0.13, 0.15], [-0.51, 0.62]])

# Assuming output size and initial weights and biases based on your description
output_size = 2
weights_fc = np.array([[0.61, 0.82, 0.96, -1.0],
                       [0.02, -0.5, 0.23, 0.17]])
biases_fc = np.array([0.25, -0.15])

learning_rate = 0.5

class NeuralNetwork:
    def __init__(self, kernel, weights_fc, biases_fc):
        # Initialize kernel and fully connected layer weights and biases
        self.kernel = kernel
        self.weights_fc = weights_fc
        self.biases_fc = biases_fc
    
    def convolve(self, image):
        # Perform convolution
        image_height, image_width = image.shape
        kernel_height, kernel_width = self.kernel.shape
        
        convolved_height = image_height - kernel_height + 1
        convolved_width = image_width - kernel_width + 1
        
        convolved = np.zeros((convolved_height, convolved_width))
        
        for i in range(convolved_height):
            for j in range(convolved_width):
                patch = image[i:i+kernel_height, j:j+kernel_width]
                convolved[i, j] = np.sum(patch * self.kernel)
        
        return convolved
    
    def max_pooling(self, convolved):
        # Perform max pooling (assuming 2x2 pooling with stride 2)
        pooled_height = convolved.shape[0] // 2
        pooled_width = convolved.shape[1] // 2
        
        pooled = np.zeros((pooled_height, pooled_width))
        
        for i in range(pooled_height):
            for j in range(pooled_width):
                patch = convolved[2*i:2*i+2, 2*j:2*j+2]
                pooled[i, j] = np.max(patch)
        
        return pooled
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def flatten(self, pooled):
        return pooled.flatten()
    
    def forward_fc(self, flattened):
        # Forward pass through fully connected layer
        z = np.dot(flattened, self.weights_fc.T) + self.biases_fc
        return self.sigmoid(z)
    
    def backward_fc(self, flattened, y_pred):
        # Backward pass through fully connected layer
        grad_z = (y_pred - 1) * y_pred * (1 - y_pred)  # Example derivative of sigmoid with respect to z
        
        delta_weights_fc = np.outer(flattened, grad_z)
        delta_biases_fc = grad_z
        
        return delta_weights_fc, delta_biases_fc

    def update_params(self, delta_weights_fc, delta_biases_fc, learning_rate):
        # Update fully connected layer weights and biases
        self.weights_fc += learning_rate * delta_weights_fc.T  # Transpose delta_weights_fc
        self.biases_fc += learning_rate * delta_biases_fc

    def train(self, image):
        # Forward pass
        convolved = self.convolve(image)
        pooled = self.max_pooling(convolved)
        flattened = self.flatten(pooled)
        y_pred = self.forward_fc(flattened)
        
        # Backward pass
        delta_weights_fc, delta_biases_fc = self.backward_fc(flattened, y_pred)
        
        # Update weights and biases
        self.update_params(delta_weights_fc, delta_biases_fc, learning_rate)
        
        return y_pred, delta_weights_fc, delta_biases_fc

# Create an instance of the NeuralNetwork class
NN = NeuralNetwork(kernel, weights_fc, biases_fc)

# Train the network (for demonstration purposes, we're not iterating over batches here)
y_pred, delta_weights_fc, delta_biases_fc = NN.train(image)

# Print the changes in parameters
print("Changes in Fully Connected Layer Weights:")
print(delta_weights_fc)
print("\nChanges in Fully Connected Layer Biases:")
print(delta_biases_fc)

Changes in Fully Connected Layer Weights:
[[-0.01540087 -0.04055962]
 [-0.01320928 -0.03478788]
 [-0.03022894 -0.07961073]
 [-0.00942382 -0.0248185 ]]

Changes in Fully Connected Layer Biases:
[-0.04980876 -0.13117602]


fold/unfold

In [36]:
import numpy as np

# Given image and kernel
image = np.array([[0.51, 0.90, 0.88, 0.84, 0.05],
                  [0.40, 0.62, 0.22, 0.59, 0.10],
                  [0.11, 0.20, 0.74, 0.33, 0.14],
                  [0.47, 0.01, 0.85, 0.70, 0.09],
                  [0.76, 0.19, 0.72, 0.17, 0.57]])

kernel = np.array([[-0.13, 0.15], [-0.51, 0.62]])

class NeuralNetwork:
    def __init__(self, kernel):
        self.kernel = kernel
    
    def unfold(self, image, kernel_size):
        image_height, image_width = image.shape
        kernel_height, kernel_width = kernel_size
        
        patches = []
        for i in range(image_height - kernel_height + 1):
            for j in range(image_width - kernel_width + 1):
                patch = image[i:i+kernel_height, j:j+kernel_width]
                patches.append(patch.flatten())
                
        return np.array(patches)
    
    def fold(self, patches, output_shape):
        output_height, output_width = output_shape
        return patches.reshape(output_height, output_width)
    
    def convolve(self, image):
        kernel_height, kernel_width = self.kernel.shape
        
        patches = self.unfold(image, self.kernel.shape)
        convolved = np.dot(patches, self.kernel.flatten())
        
        convolved_height = image.shape[0] - kernel_height + 1
        convolved_width = image.shape[1] - kernel_width + 1
        
        convolved = self.fold(convolved, (convolved_height, convolved_width))
        
        return convolved

# Create an instance of the NeuralNetwork class
NN = NeuralNetwork(kernel)

# Perform convolution
convolved = NN.convolve(image)

print("Convolved Image:")
print(convolved)

Convolved Image:
[[ 0.2491 -0.1648  0.2652 -0.3406]
 [ 0.1089  0.3092 -0.1129 -0.1432]
 [-0.2178  0.6069 -0.0462 -0.3231]
 [-0.3294  0.4757 -0.2673  0.1892]]
